# Read in data

In [30]:
if (!require("tidyverse")) install.packages("tidyverse")
#if (!require("VIM")) install.packages("VIM")
#if (!require("Amelia")) install.packages("Amelia")
#if (!require("naniar")) install.packages("naniar")
#if (!require("visdat")) install.packages("visdat")
#if (!require("SparseM")) install.packages("SparseM")
#if (!require("DataExplorer")) install.packages("DataExplorer")
if (!require("janitor")) install.packages("janitor")
#if (!require("tidySEM")) install.packages("tidySEM")
#if (!require("semPlot")) install.packages("semPlot")
#if (!require("lavaan")) install.packages("lavaan")
#if (!require("lavaanPlot")) install.packages("lavaanPlot")
#if (!require("pracma")) install.packages("pracma")  
library(R.matlab)
library(psych)
if (!require("officer")) install.packages("officer")


Loading required package: officer



In [4]:
library(purrr)
library(dplyr)
library(childsds)
library(R.matlab)


In [5]:
data_list=list()

files=c('abcd_ant01.txt', 'abcd_ssphy01.txt', 'abcd_lt01.txt', 'abcd_bkfs01.txt', 'abcd_rhds01.txt',
       'abcd_nsc01.txt', 'pdem02.txt', 'abcd_yrb01.txt', 'abcd_mhy02.txt', 'cct01.txt', 'abcd_yddss01.txt',
       'abcd_ps01.txt', 'abcd_yest01.txt', 'abcd_tbss01.txt', 'lmtp201.txt', 'abcd_ybd01.txt',
       'abcd_bp01.txt', 'abcd_dmriqc01.txt', 'abcd_fsurfqc01.txt','abcd_smrip10201.txt','abcd_dti_p101.txt',
       'abcd_lpds01.txt','acspsw03.txt')

for (i in 1:length(files)){
    data = readLines(paste("/dagher/dagher9/filip/ABCD/Package_1219524/",files[i],sep=''))
    skip_second = data[-2]
    data = read.csv(textConnection(skip_second), header = TRUE, stringsAsFactors = FALSE, sep='\t')
    drops <- c("dataset_id", "interview_date",'collection_id','src_subject_id')
    data = data[ , !(names(data) %in% drops)]    
    data_list[[i]]=data
}

pcs=read.table('/dagher/dagher11/filip/Downloads/ABCD_pre-impute.eigenvec', header=T)
prs=read.table('/dagher/dagher11/filip/Downloads/BMI_PRS_new.csv',header=T, sep=',')


colnames(pcs)=c('FID','subjectkey',colnames(pcs)[3:ncol(pcs)])
colnames(prs)=c('X','FID2','subjectkey','PRS','Z')
data_list[[i+1]]=pcs
data_list[[i+2]]=prs

In [6]:
dataset = data_list %>% reduce(full_join)

Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, interview_age, sex, eventname, collection_title, study_cohort_name)`
Joining with `by = join_by(subjectkey, intervie

In [7]:
dataset$BMI=(dataset$anthroweight1lb/dataset$anthroheightcalc^2)*703
dataset$age_yrs=dataset$interview_age/12
dataset$BMI_SD=sds(value = dataset$BMI, age = dataset$age_yrs, sex = dataset$sex, 
                   male = "M", female = "F",ref = cdc.ref, item = "bmi",type = "SDS")


In [8]:
dataset=(dataset[dataset$eventname=='baseline_year_1_arm_1',])#Remove some follow-up periods that we will not be using


In [9]:
dataset=dataset[!(dataset$BMI>50 | dataset$BMI<10),]

In [10]:
dataset$BMI_SD[is.infinite(dataset$BMI_SD)] <- NA

In [11]:
siblings=dataset[duplicated(dataset$rel_family_id, incomparables=NA),]$subjectkey
nrow(dataset)
dataset=dataset[!dataset$subjectkey %in% siblings,]
nrow(dataset)

[1] 11849

[1] 9837

In [12]:
dataset=dataset[(dataset$fsqc_qc==1 | is.na(dataset$fsqc_qc)),] # important because there are NA values for QC for 1 year f-up

In [21]:
dataset[dataset == 999] <- NA
dataset[dataset == 777] <- NA

In [53]:
write.table(dataset,'/dagher/dagher11/filip/Obesity_maps/data/dataset_ABCD.csv',sep=',',row.names=F)

In [33]:
results=matrix(ncol=6,nrow=length(c(grep('thick_cdk',colnames(dataset))))) 

for (i in 1:length(c(grep('thick_cdk',colnames(dataset))))) 
{ 
    
    m1=summary(lm(dataset[c(grep('thick_cdk',colnames(dataset)))][[i]] ~ 
          dataset$BMI_SD +
          dataset$age_yrs +
          dataset$sex + 
          dataset$site_id_l 
          #dataset$demo_prnt_ed_v2 +
          #dataset$demo_comb_income_v2
                 ))
    
    results[i,1]=colnames(dataset)[c(grep('thick_cdk',colnames(dataset)))][i] 
    results[i,2]=m1$coefficients[2,4] # p-value 
    results[i,3]=m1$coefficients[2,3] # t-value 
    results[i,4]=m1$coefficients[2,1] # sd for 
    results[i,5]=m1$coefficients[2,2] # estimate for 
    results[i,6]=ifelse(grepl("left",results[i,1]),"left","right") #save hemisphere 

}

colnames(results)=c('Parcel','p-value', 't-value', 'SD', 'estimate', 'hemisphere')

In [24]:
write.table(results,'/dagher/dagher11/filip/Obesity_maps/tables/ABCD_BMI_noSES.csv',sep=',',row.names=F,col.names=T)

In [27]:
writeMat('/dagher/dagher11/filip/Obesity_maps/data/ABCD_CT_DKT_RH_noSES.mat','ABCD_CT_DKT_RH'=t(as.numeric(results[,3][c(36:64,67:68)])))
writeMat('/dagher/dagher11/filip/Obesity_maps/data/ABCD_CT_DKT_LH_noSES.mat','ABCD_CT_DKT_LH'=t(as.numeric(results[,3][c(2:30,33:34)])))

In [26]:
writeMat('/dagher/dagher11/filip/Obesity_maps/data/ABCD_CT_DKT_LHRH.mat_noSES','ABCD_CT_DKT_LHRH'=(as.numeric(results[,3][c(2:30, 33:34, 36:64,67:68)])))


In [35]:
results=as.data.frame(results)

rows_to_remove <- c(1, 31, 32, 35, 65, 67, 69, 70, 71)
results <- results[-rows_to_remove, ]

colnames(results)=c('Parcel','p-value', 't-value', 'SD', 'estimate', 'hemisphere')

results$'p-value'=as.numeric(results$'p-value')
results$'t-value'=as.numeric(results$'t-value')
results$SD=as.numeric(results$SD)
results$estimate=as.numeric(results$estimate)
results$Parcel=c("Caudal Anterior Cingulate", "Caudal Middle Frontal",
"Cuneus", "Entorhinal",
"Fusiform", "Inferior Parietal",
"Inferior Temporal", "Isthmus Cingulate",
"Lateral Occipital", "Lateral Orbitofrontal",
"Lingual", "Medial Orbitofrontal",
"Middle Temporal", "Parahippocampal",
"Paracentral", "Pars Opercularis",
"Pars Orbitalis", "Pars Triangularis",
"Pericalcarine", "Postcentral",
"Posterior Cingulate", "Precentral",
"Precuneus", "Rostral Anterior Cingulate",
"Rostral Middle Frontal", "Superior Frontal",
"Superior Parietal", "Superior Temporal",
"Supramarginal", "Transverse Temporal",
"Insula", "Caudal Anterior Cingulate",
"Caudal Middle Frontal", "Cuneus",
"Entorhinal", "Fusiform",
"Inferior Parietal", "Inferior Temporal",
"Isthmus Cingulate", "Lateral Occipital",
"Lateral Orbitofrontal", "Lingual",
"Medial Orbitofrontal", "Middle Temporal",
"Parahippocampal", "Paracentral",
"Pars Opercularis", "Pars Orbitalis",
"Pars Triangularis", "Pericalcarine",
"Postcentral", "Posterior Cingulate",
"Precentral", "Precuneus",
"Rostral Anterior Cingulate", "Rostral Middle Frontal",
"Superior Frontal", "Superior Parietal",
"Superior Temporal", "Supramarginal",
"Transverse Temporal", "Insula")
results$hemisphere=NULL


doc <- read_docx()
results[, sapply(results, is.numeric)] <- round(results[, sapply(results, is.numeric)], 4)
doc <- doc %>%
  body_add_table(as.data.frame(results))

print(doc, target = "/dagher/dagher11/filip/Obesity_maps/tables/ABCD_BMInoSES.docx")

In [41]:
library(psych)

In [19]:
describe(dataset$age_yrs)
describe(dataset$BMI)
table(dataset$sex)
describe(dataset$BMI_SD)


,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X1,1,9521,9.90724,0.6184078,9.916667,9.897608,0.86485,8.916667,11,2.083333,0.07181769,-1.249936,0.006337729


,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X1,1,9521,18.91856,4.218981,17.74372,18.3291,3.131453,10.49254,45.61689,35.12435,1.513433,2.998619,0.04323807



   F    M 
4543 4978 

,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X1,1,9521,0.4490158,1.175252,0.4624532,0.4819788,1.225747,-6.968164,3.084819,10.05298,-0.4207062,0.6568662,0.01204452


In [43]:
nrow(dataset)

[1] 9533